# Tests

In [1]:
# Check evalf works properly
!python test_evalf.py

Passed the manual regression test
Passed the steady-state test
Passed the constant-speed state test (1000 trials)
Passed the acceleration test (1000 trials)
Passed the spinning test (1000 trials)
Passed the level turn test (1000 trials)


In [2]:
# Check jacobian works properly
!python test_jacobian.py

  File "/Users/youngjinpark/6.7300-project-ev/test_jacobian.py", line 62
    J_ANALYTIC = J_analytic_no_pos]
                                  ^
SyntaxError: unmatched ']'


In [3]:
# Compare our Jax Jacobian vs. Finite Difference
!python test_jacobian_fd.py

Configuration: OUTPUT_POSITIONS = False
Number of states: 8

JACOBIAN TEST RESULTS

✓ PASSED: Manual Regression (2 trials) [regression]
  Max Error: 0.00e+00
  Avg Error: 0.00e+00

✓ PASSED: Steady State [finite_difference]
  Max Error: 9.93e-12
  Avg Error: 1.55e-13

✓ PASSED: Constant Speed (1000 trials) [finite_difference]
  Max Error: 6.47e-06
  Avg Error: 2.14e-06

✓ PASSED: Acceleration (1000 trials) [finite_difference]
  Max Error: 1.60e-04
  Avg Error: 5.80e-05

✓ PASSED: Spinning (1000 trials) [finite_difference]
  Max Error: 5.84e-11
  Avg Error: 2.21e-11

✓ PASSED: Level Turn (1000 trials) [finite_difference]
  Max Error: 6.96e-06
  Avg Error: 2.17e-06

Overall: ALL TESTS PASSED
Total Trials: 4003
Overall Average Error: 1.04e-05


# Detailed Tutorials

In [4]:
import jax
jax.config.update("jax_enable_x64", True)

import jax.numpy as jnp
import numpy as np

from vehicle_model_jax import evalf, evalf_np, compute_jacobian_jax, get_default_params, OUTPUT_POSITIONS, N_STATES, IDX_V

## Get Default Params

In [5]:
p = get_default_params()  # dictionary

p

{'m': 1500,
 'I': 2500,
 'c_d': 25,
 'c_r': 500,
 'R_s': 0.1,
 'L_ds': 0.002,
 'L_qs': 0.003,
 'lambda_f': 0.5,
 'p_pairs': 4,
 'G': 10.0,
 'r_w': 0.3,
 'eta_g': 0.95,
 'k': 31.666666666666668,
 'K_p_i': 10.0,
 'K_i_i': 100.0,
 'K_p_v': 100.0,
 'K_i_v': 100.0,
 'K_p_theta': 1000.0,
 'K_d_theta': 1000.0,
 'v_w': 0.0,
 'psi': 0.0,
 'c_wx': 0.5,
 'c_wy': 0.5}

In [6]:
p_tuple = tuple(p.values())  # tuple

p_tuple

(1500,
 2500,
 25,
 500,
 0.1,
 0.002,
 0.003,
 0.5,
 4,
 10.0,
 0.3,
 0.95,
 31.666666666666668,
 10.0,
 100.0,
 100.0,
 100.0,
 1000.0,
 1000.0,
 0.0,
 0.0,
 0.5,
 0.5)

## How to get f(x, u)

### Using Jax

In [7]:
# Initialize the state, x0
x0 = jnp.zeros(N_STATES)  # note that you should use jax numpy array
x0 = x0.at[IDX_V].set(1.0)

# Initialize the input, u
u = jnp.array([1.0, 0.0])

f_jax = evalf(x0, p_tuple, u)
    
f_jax

Array([ 0.00000000e+00, -5.55555556e+03, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -1.70000000e-02,  0.00000000e+00,  0.00000000e+00],      dtype=float64)

In [8]:
f_jax.shape

(8,)

### Using Numpy

In [9]:
x0_np = np.zeros(N_STATES)
x0_np[IDX_V] = 1.0
x0_np = x0_np[:, None]  # note that you should make (N, 1) matrix when using evalf_np

u_np = np.array([1.0, 0.0])

f_np = evalf_np(x0_np, p_tuple, u_np)

f_np

Array([[ 0.00000000e+00],
       [-5.55555556e+03],
       [-0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [-1.70000000e-02],
       [ 0.00000000e+00],
       [ 0.00000000e+00]], dtype=float64)

In [10]:
f_np.shape  # note that the output shape is also (N, 1) matrix when using evalf_np

(8, 1)

## How to get Jacobian?

In [11]:
J = compute_jacobian_jax(x0, p_tuple, u)

J

Array([[-5.05000000e+03,  5.00000000e+01,  5.00000000e+04,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-2.22222222e+01, -3.36666667e+03,  0.00000000e+00,
         3.33333333e+04,  0.00000000e+00, -9.06432749e+03,
         0.00000000e+00,  3.50877193e+03],
       [-1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.05263158e+00,
         0.00000000e+00,  1.05263158e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  6.33333333e-02,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.73333333e-02,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.000000

In [12]:
J.shape

(8, 8)

In [13]:
condition_number = np.linalg.cond(J)
print("Condition number:", condition_number)

Condition number: 1096281.9446778735


In [14]:
np.linalg.inv(J)

array([[ 1.61972664e-20,  2.02671505e-22, -1.00000000e+00,
        -2.09484992e-19, -0.00000000e+00,  2.04017221e-17,
        -0.00000000e+00, -2.50624604e-18],
       [-1.37168975e-22,  3.13918774e-20, -4.52330507e-19,
        -3.26113403e-18, -0.00000000e+00,  1.57894737e+01,
        -0.00000000e+00, -2.73684211e-01],
       [ 2.00000000e-05, -1.09220554e-23, -1.01000000e-01,
        -1.78968501e-20, -0.00000000e+00, -1.57894737e-02,
        -0.00000000e+00,  2.73684211e-04],
       [-2.20673418e-23,  3.00000000e-05, -6.66666667e-04,
        -1.00000000e-01, -0.00000000e+00,  1.57894737e-02,
        -0.00000000e+00, -1.66940351e-01],
       [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -1.50000000e+00, -0.00000000e+00,
        -2.50000000e+00, -0.00000000e+00],
       [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -1.00000000e+00],
       [ 0.00000000e+00,  0.000000